In [56]:
import pandas as pd
from tqdm import tqdm

In [57]:
# TODO house_interaction에 클러스터 결과를 merge해줍니다. 이때, 클러스터 그룹이 -1이거나 house_interaction에만 포함된 item들은 cluster_id.max() + 1을 더해줍니다.
# TODO model을 학습해서 학습한 결과를 backend에 반영합니다.
# TODO cluster-major-item list를 생성합니다.

In [58]:
item = pd.read_csv("/opt/ml/input/final/EDA/clustered_item_32715.csv")
train = pd.read_csv("/opt/ml/input/data/hi_interaction.csv")

In [59]:
item.head(1)

,item,category,rating,review,price,preprocessed_title,seller,discount_rate,image,available_product,predict_price,original_title,similar_item_list,cls_id
0,1,가구|서랍·수납장|수납장,별점 4.2점,(67)개 리뷰,NaN,66808,DRAWER,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,예상가정보없음사이즈59.0 × 37.0 × 90.0 (cm)출처www.casa.co.kr,66808,NaN,-1


In [60]:
train.head(1)

,house,item
0,318,55


In [61]:
item.cls_id.nunique()

8740

In [62]:
item.item.nunique()

88634

In [63]:
len(train)

412784

In [64]:
# item에 포함되지 않은 hi_interaction item을 제거하기
train = train[train.item.isin(item.item.unique())]

In [65]:
len(train)

257305

In [66]:
item.cls_id.max()

9190

In [12]:
len(item[item.cls_id == -1])

63595

In [13]:
dummy = list()
c = item.cls_id.max()

for i in range(len(item[item.cls_id == -1])):
    dummy.append(i + c + 1)

In [14]:
item.loc[item.cls_id == -1, "cls_id"] = dummy

In [15]:
item.head(3)

,item,category,rating,review,price,preprocessed_title,seller,discount_rate,image,available_product,predict_price,original_title,similar_item_list,cls_id
0,1,가구|서랍·수납장|수납장,별점 4.2점,(67)개 리뷰,NaN,66808,DRAWER,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,예상가정보없음사이즈59.0 × 37.0 × 90.0 (cm)출처www.casa.co.kr,66808,NaN,9191
1,4,데코·식물|시계|벽시계,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,118400,우드피아,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,예상가정보없음사이즈22.0 × 22.0 × 1.5 (cm)출처www.11st.co.kr,118400,NaN,9192
2,8,가구|거실장·TV장|일반거실장,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,131836,DOICH(도이치),NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,예상가정보없음사이즈165.0 × 40.0 × 37.5 (cm)출처m.doich.co.kr,131836,NaN,9193


In [ ]:
item.cls_id.nunique()

In [16]:
item2cls = {item_id:cls_id for item_id, cls_id in zip(item.item, item.cls_id)}

In [17]:
train["cluster_id"] = train.item.map(item2cls)

In [18]:
train

,house,item,cluster_id
0,318,55,0
1,318,1490,17
4,318,2256,10067
5,318,2257,10068
6,318,2258,10069
...,...,...,...
412761,135273,1169670,65113
412764,135273,1451760,69894
412765,135273,1467566,70090
412767,135273,1709921,72269


In [19]:
train = train[["house", "cluster_id", "item"]]

In [20]:
train = train.sort_values(by=["house", "cluster_id", "item"])

In [21]:
train

,house,cluster_id,item
0,318,0,55
1,318,17,1490
4,318,10067,2256
5,318,10068,2257
6,318,10069,2258
...,...,...,...
412761,135273,65113,1169670
412764,135273,69894,1451760
412765,135273,70090,1467566
412767,135273,72269,1709921


In [22]:
train.to_csv("clustered_train.csv", index=False)

In [23]:
predict_price_list = []

import re
p = re.compile('.+(?=사)')
for i in item.predict_price.index:
    m = p.findall(str(item.predict_price.iloc[i]))
    if len(m) == 0:
        m.append("예상가정보없음")
    if m[0][3:] == "정보없음":
        predict_price_list.append("정보없음")
    else:
        predict_price_list.append(int("".join((m[0][3:]).split(","))))

In [24]:
item.predict_price = predict_price_list

In [25]:
item.loc[(item.predict_price != "정보없음"), "price"] = item[(item.predict_price != "정보없음")].predict_price

In [27]:
import re
def f(x):
    pattern = re.compile(r"별점\s+(\d+\.\d+)점")
    try:
        match = pattern.search(x)
        if match:
            score = float(match.group(1))
        else:
            score = None
        return score
    except:
        return 0.0

In [28]:
item.rating = item.rating.apply(lambda x: f(x))

In [29]:
item.review = item.review.fillna("리뷰 쓰기첫 리뷰 두 배 적립")
review_list = []

import re
p = re.compile('[(][0-9]*[)]')
for idx, review in enumerate(item.review):
    m = p.findall(review)
    if len(m) == 0:
        m.append("(0)")
    review_list.append(m[0][1:-1])

item.review = review_list

In [30]:
item.review = item.review.astype(int)

In [31]:
item.rating = item.rating.astype(float)

In [32]:
item.head(5)

,item,category,rating,review,price,preprocessed_title,seller,discount_rate,image,available_product,predict_price,original_title,similar_item_list,cls_id
0,1,가구|서랍·수납장|수납장,4.2,67,NaN,66808,DRAWER,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,66808,NaN,9191
1,4,데코·식물|시계|벽시계,0.0,0,NaN,118400,우드피아,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,118400,NaN,9192
2,8,가구|거실장·TV장|일반거실장,0.0,0,NaN,131836,DOICH(도이치),NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,131836,NaN,9193
3,9,가구|의자|스툴·벤치,5.0,2,NaN,204593,이케아,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,204593,NaN,9194
4,10,가구|화장대·콘솔|일반화장대,0.0,0,NaN,221527,아노말리,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,정보없음,221527,NaN,9195


In [33]:
item = item.sort_values(["cls_id", "review", "rating"], ascending=[True, False, False])

In [35]:
item.groupby("cls_id")["item"].apply(lambda x:len(list(x)))

cls_id
0        22
1         3
2         2
3         2
4         2
         ..
72781     1
72782     1
72783     1
72784     1
72785     1
Name: item, Length: 72334, dtype: int64

<!-- 1. 클러스터링 적당히 엄격히하고 (현 수준) -> 생성된 테이블로 백, 프론트, DB 작업하고 -> 검증만 다중선형회귀로
1. AE 학습할 때 feature로 넣는다? -->

In [46]:
cls_major_item_id = item.rename(columns={"item":"major_item"}).groupby("cls_id")["major_item"].apply(lambda x:list(x)[0]).reset_index()
cls_major_item_id_list = item.rename(columns={"item":"item_list"}).groupby("cls_id")["item_list"].apply(lambda x:"|".join(list(map(str, list(x))))).reset_index()

In [47]:
cls_major_item_id = cls_major_item_id.merge(cls_major_item_id_list, on="cls_id")

In [48]:
cls_major_item_id

,cls_id,major_item,item_list
0,0,456458,456458|456462|55|456449|138899|133699|769|3387...
1,1,1017200,1017200|743437|89
2,2,103,103|350421
3,3,107,107|1436
4,4,108,108|143
...,...,...,...
72329,72781,1820832,1820832
72330,72782,1821165,1821165
72331,72783,1821312,1821312
72332,72784,1822506,1822506


In [49]:
cls_major_item_id.to_csv("cluster_major_item.csv", index=True)